# Init

In [63]:
! cd ..

/Users/wliao0504/code/apt-plus


/Users/wliao0504/code/apt-plus/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


## Import

In [130]:
import duckdb
import pandas as pd
from openai import OpenAI
import json
import re
from importlib import reload

In [134]:
import scripts.inspections_utils as iu
reload(iu)
from scripts.inspections_utils import (
    instruction,
    filter_by_violation_code,
    generate_summary_of_trivial_violations,
    filter_df_by_address,
    generate_prompt_from_address,
    generate_narrative_report_for_one_occasion,
    generate_narrative_report_for_all_occasions,
    remove_trivial_violations_by_code,
    TRIVIAL_VIOLATION_CODES,
    BASE_PROMPT,
    ADDRESSES_TO_CHECK
)

## Load data

In [51]:
df_all = pd.read_csv("../data/Building_Violations_20250421.csv")

In [52]:
query = """
SELECT 
    ADDRESS
    ,"ID" as "VIOLATION ID"
    ,strptime("VIOLATION DATE", '%m/%d/%Y') as "VIOLATION DATE"
    ,"DEPARTMENT BUREAU"
    ,"INSPECTION NUMBER" as "INSPECTION ID"
    ,"INSPECTION CATEGORY"
    ,"INSPECTION STATUS"
    , "INSPECTOR ID"
    ,"VIOLATION ORDINANCE"
    ,"VIOLATION CODE"
    ,"VIOLATION DESCRIPTION"
    ,"VIOLATION LOCATION"
    ,"VIOLATION INSPECTOR COMMENTS"
    ,"VIOLATION STATUS" 
    , "VIOLATION STATUS DATE"
FROM df_all
ORDER BY ADDRESS, "VIOLATION DATE" desc, "INSPECTION CATEGORY"
"""
df_all = duckdb.sql(query).df()
df_all.head()

,ADDRESS,VIOLATION ID,VIOLATION DATE,DEPARTMENT BUREAU,INSPECTION ID,INSPECTION CATEGORY,INSPECTION STATUS,INSPECTOR ID,VIOLATION ORDINANCE,VIOLATION CODE,VIOLATION DESCRIPTION,VIOLATION LOCATION,VIOLATION INSPECTOR COMMENTS,VIOLATION STATUS,VIOLATION STATUS DATE
0,1 E 113TH ST,6123817,2018-10-19,VENTILATION,12720654,PERMIT,PASSED,203779,Arrange mechanical ventilation or warm air hea...,VT1010,ARRANGE COMPLETED PERMIT INSP.,None,None,COMPLIED,12/12/2018
1,1 E 113TH ST,3997253,2011-10-12,ELECTRICAL,10349861,COMPLAINT,PASSED,536640,Remove exposed non-metallic cable. (18-27-336....,EL0059,REMOVE EXPOSED CABLE,None,ROMEX ABOVE DROP CEILING IN REAR,COMPLIED,12/20/2011
2,1 E 113TH ST,3997242,2011-10-12,ELECTRICAL,10349861,COMPLAINT,PASSED,536640,Install system II emergency lighting. (18-27-7...,EL0026,SYSTEM II,None,PROVIDE BATTERY BACK-UP EXITS SIGNS,COMPLIED,12/20/2011
3,1 E 113TH ST,3997243,2011-10-12,ELECTRICAL,10349861,COMPLAINT,PASSED,536640,Restore exit and emergency lighting system and...,EL0027,RESTORE EXIT AND EM,None,None,COMPLIED,12/20/2011
4,1 E 113TH ST,3997244,2011-10-12,ELECTRICAL,10349861,COMPLAINT,PASSED,536640,Remove unapproved wiring and/or equipment from...,EL0028,UNAPPROVED EM,None,PLASTIC HOUSING FOR EM & EXIT NOT ALLOWED,COMPLIED,12/20/2011


In [53]:
HP_NORTH_BOUND = 41.809647
HP_SOUTH_BOUND = 41.780482
HP_WEST_BOUND = -87.615877
HP_EAST_BOUND = -87.579056

query = f"""
SELECT *
FROM df_all
WHERE "VIOLATION DATE" >= DATE '2020-01-01'
-- AND LATITUDE BETWEEN {HP_SOUTH_BOUND} AND {HP_NORTH_BOUND}
-- AND LONGITUDE BETWEEN {HP_WEST_BOUND} AND {HP_EAST_BOUND}
"""
df_recent = duckdb.sql(query).df()
df_recent.head()

,ADDRESS,VIOLATION ID,VIOLATION DATE,DEPARTMENT BUREAU,INSPECTION ID,INSPECTION CATEGORY,INSPECTION STATUS,INSPECTOR ID,VIOLATION ORDINANCE,VIOLATION CODE,VIOLATION DESCRIPTION,VIOLATION LOCATION,VIOLATION INSPECTOR COMMENTS,VIOLATION STATUS,VIOLATION STATUS DATE
0,1 E 114TH ST,6474286,2020-06-15,CONSERVATION,13282944,COMPLAINT,FAILED,BL01000,Failed to repair or replace defective or missi...,CN070024,REPAIR PORCH SYSTEM,EXTERIOR:S :,REAR PORCH - FOUNDATION STATUS UNKNOWN; BROKEN...,OPEN,None
1,1 E 114TH ST,6474277,2020-06-15,CONSERVATION,13282944,COMPLAINT,FAILED,BL01000,Repair or replace broken or defective plumbing...,PL159017,REPLACE DEFECTIVE PLBG FIXTURE,INTERIOR:001 :HOUSE,"REAR BATHROOM - TOILET NOT WORKING, CLOGGED AT...",OPEN,None
2,1 E 114TH ST,6474282,2020-06-15,CONSERVATION,13282944,COMPLAINT,FAILED,BL01000,"Failed to cut or remove weeds, grass or other ...",CN041063,CUT OR REMOVE HIGH WEEDS,EXTERIOR:S :,"TALL TREES, BUSHES OVERHNAGING REAR EXIT - OBS...",OPEN,None
3,1 E 114TH ST,6474281,2020-06-15,CONSERVATION,13282944,COMPLAINT,FAILED,BL01000,Failed to maintain roof downspouts in good rep...,CN076044,REPAIR DOWNSPOUT,EXTERIOR:E :,DOWNSPOUT - SECTION MISSING,OPEN,None
4,1 E 114TH ST,6474279,2020-06-15,CONSERVATION,13282944,COMPLAINT,FAILED,BL01000,Failed to maintain floor free from holes and w...,CN103015,REPAIR FLOOR,INTERIOR:001 :HOUSE,"LIVINGROOM, KITCHEN/FLOOR - SOFT VOIDS IN FLOOR",OPEN,None


In [54]:
df_periodic = df_all[df_all["INSPECTION CATEGORY"].eq("PERIODIC")]
df_periodic.head()

,ADDRESS,VIOLATION ID,VIOLATION DATE,DEPARTMENT BUREAU,INSPECTION ID,INSPECTION CATEGORY,INSPECTION STATUS,INSPECTOR ID,VIOLATION ORDINANCE,VIOLATION CODE,VIOLATION DESCRIPTION,VIOLATION LOCATION,VIOLATION INSPECTOR COMMENTS,VIOLATION STATUS,VIOLATION STATUS DATE
44,1 E 119TH ST,3415032,2010-02-25,CONSERVATION,9710833,PERIODIC,FAILED,BL00444,"Post name, address, and telephone of owner, ow...",CN196029,POST OWNER/MANAGERS NAME/#,OTHER : :,OWNERSHIP MISSING,OPEN,None
45,1 E 119TH ST,3415027,2010-02-25,CONSERVATION,9710833,PERIODIC,FAILED,BL00444,Failed to maintain exterior door in sound cond...,CN073014,REPAIR EXTERIOR DOOR,OTHER : :,WEST EXTERIOR DOOR BENT,OPEN,None
46,1 E 119TH ST,3415029,2010-02-25,CONSERVATION,9710833,PERIODIC,FAILED,BL00444,"Replace broken, missing or defective window pa...",CN104015,"REPLCE WINDOW PANES, PLEXGLAS",OTHER : :,NORTH ELEVATION 2ND FLR.WINDOW PANE BROKEN,OPEN,None
47,1 E 119TH ST,3415025,2010-02-25,CONSERVATION,9710833,PERIODIC,FAILED,BL00444,Failed to remove obstruction from exitway that...,CN015062,OBSTRUCTIONS IN EXIT WAY,OTHER : :,EAST AND WEST GATE PADLOCK,OPEN,None
48,1 E 119TH ST,3415030,2010-02-25,CONSERVATION,9710833,PERIODIC,FAILED,BL00444,Repair or replace defective door. (13-196-550),CN105015,"REPAIR DOOR, INT.",OTHER : :,FOYER BASEMENT DOOR BROKEN,OPEN,None


### recent periodic and complaint

In [55]:
df_recent_subset = df_recent[df_recent["INSPECTION CATEGORY"].isin(["PERIODIC", "COMPLAINT"])]

# EDA

### Inspection Category

In [24]:
df_all.value_counts("INSPECTION CATEGORY")

INSPECTION CATEGORY
COMPLAINT       1377449
PERIODIC         475334
PERMIT            98689
REGISTRATION       2593
Name: count, dtype: int64

### Violation Code

check if violation code -> violation description mapping is one-to-one. turns out one code would only map to one description, but one description might map to multiple codes.

In [10]:
query = """
SELECT 
    "VIOLATION CODE",
FROM df_all
GROUP BY "VIOLATION CODE"
HAVING COUNT(DISTINCT "VIOLATION DESCRIPTION") > 1
"""
duckdb.sql(query).df()

,VIOLATION CODE


In [11]:
query = """
SELECT 
    "VIOLATION DESCRIPTION",
FROM df_all
GROUP BY "VIOLATION DESCRIPTION"
HAVING COUNT(DISTINCT "VIOLATION CODE") > 1
"""
duckdb.sql(query).df()

,VIOLATION DESCRIPTION
0,SCOPE OF WORK ON APPLICATION C
1,ACCESS TO JBS
2,ROOF DRAINAGE 29 [ 18-29-1101.
3,FIRE & WATER DAMAGE
4,SEE INSPECTOR COMMENTS
...,...
105,DEFECTIVE OUTLETS
106,CLOSE BOX OPENINGS
107,NO ENTRY
108,DEFECTIVE LIGHTING SERVICE


In [12]:
source_df = "df_recent_subset"
query = f"""
WITH violation_counts AS (
  SELECT
    "VIOLATION DESCRIPTION",
    "VIOLATION CODE",
    COUNT(DISTINCT "VIOLATION CODE") OVER (PARTITION BY "VIOLATION DESCRIPTION") AS n_code_per_desc,
    COUNT(*) OVER (PARTITION BY "VIOLATION DESCRIPTION", "VIOLATION CODE") AS n_violations
  FROM {source_df}
),
prioritized AS (
  SELECT *,
         MAX(n_violations) OVER (PARTITION BY "VIOLATION DESCRIPTION") AS max_desc_violations
  FROM violation_counts
),
grouped_ranked AS (
  SELECT *,
         DENSE_RANK() OVER (ORDER BY max_desc_violations DESC) AS group_priority
  FROM prioritized
)
SELECT DISTINCT
  "VIOLATION DESCRIPTION",
  "VIOLATION CODE",
  n_code_per_desc,
  n_violations
FROM grouped_ranked
ORDER BY group_priority, "VIOLATION DESCRIPTION", n_violations DESC
"""
code_to_description = duckdb.sql(query).df()

In [13]:
code_to_description[code_to_description["VIOLATION DESCRIPTION"] == 'ARRANGE PREMISE INSPECTION']

,VIOLATION DESCRIPTION,VIOLATION CODE,n_code_per_desc,n_violations
0,ARRANGE PREMISE INSPECTION,CN190019,2,14727
1,ARRANGE PREMISE INSPECTION,CN193305,2,3


In [15]:
df_all["VIOLATION CODE"].value_counts()

VIOLATION CODE
CN190019       104670
EV1110          68708
CN061014        63171
CN196029        57834
CN070024        50343
                ...  
EV0325              1
NC1495              1
FD CIT 04-C         1
RN1697              1
HSF20               1
Name: count, Length: 1566, dtype: int64

In [16]:
df_all["VIOLATION DESCRIPTION"].value_counts()

VIOLATION DESCRIPTION
ARRANGE PREMISE INSPECTION        104682
MAINTAIN OR REPAIR ELECT ELEVA     68708
REPAIR EXTERIOR WALL               63171
POST OWNER/MANAGERS NAME/#         57834
REPAIR PORCH SYSTEM                50343
                                   ...  
REP/REPL DEF DR CLOSE BUTTON P         1
VENT CAR ENCLOSURE PASS                1
REP/REPL CAR/TOP OPER/DEVICE F         1
PRVD UNOBST EGRESS F/VCNTCOMM          1
INSECTS AND VERMIN CONTROL             1
Name: count, Length: 1396, dtype: int64

In [17]:
violation_code_sorted = df_all.sort_values("VIOLATION CODE")
violation_code_sorted.head()

,ADDRESS,VIOLATION ID,VIOLATION DATE,DEPARTMENT BUREAU,INSPECTION ID,INSPECTION CATEGORY,INSPECTION STATUS,VIOLATION ORDINANCE,VIOLATION CODE,VIOLATION DESCRIPTION,VIOLATION LOCATION,VIOLATION INSPECTOR COMMENTS,VIOLATION STATUS,VIOLATION STATUS DATE
755331,3220 W 26TH ST,2925793,2009-01-23,NEW CONSTRUCTION,2667341,PERMIT,PASSED,None,002011,NO PLANS OR PERMITS ON SITE,None,None,COMPLIED,03/05/2009
1084422,4722 W FULLERTON AVE,3255025,2009-08-12,REFRIGERATION,2797413,PERIODIC,FAILED,None,002011,NO PLANS OR PERMITS ON SITE,None,FOR FOUR 5 TON SELF CONTAINED GOODMAN A/C'S ON...,OPEN,None
1549118,6740 S CLYDE AVE,5410933,2016-05-24,VENTILATION,11935781,PERMIT,FAILED,None,002011,NO PLANS OR PERMITS ON SITE,None,None,OPEN,None
781780,3321 S OAKLEY AVE,2021546,2007-05-23,IRON,1902095,COMPLAINT,FAILED,None,002011,NO PLANS OR PERMITS ON SITE,None,SUBMIT PLANS AND PERMIT FOR NEW STEEL PORCH,OPEN,None
709980,304 W 71ST ST,2863724,2008-12-12,ELECTRICAL,2709635,PERMIT,PASSED,None,002011,NO PLANS OR PERMITS ON SITE,None,None,COMPLIED,05/08/2009


### Inspection Status

check if the inspection id would always get the same inspecton status

In [22]:
query = """
SELECT 
    "INSPECTION ID",
    COUNT(DISTINCT "INSPECTION STATUS") as n_status
FROM df_all
GROUP BY "INSPECTION ID"
ORDER BY n_status DESC 
"""
duckdb.sql(query).df()

BinderException: Binder Error: Referenced column "INSPECTION NUMBER" not found in FROM clause!
Candidate bindings: "INSPECTION ID", "INSPECTION CATEGORY", "INSPECTION STATUS", "VIOLATION INSPECTOR COMMENTS", "VIOLATION ORDINANCE"

In [16]:
df_all[df_all["INSPECTION NUMBER"] == 11219187]

,ID,VIOLATION LAST MODIFIED DATE,VIOLATION DATE,VIOLATION CODE,VIOLATION STATUS,VIOLATION STATUS DATE,VIOLATION DESCRIPTION,VIOLATION LOCATION,VIOLATION INSPECTOR COMMENTS,VIOLATION ORDINANCE,...,ADDRESS,STREET NUMBER,STREET DIRECTION,STREET NAME,STREET TYPE,PROPERTY GROUP,SSA,LATITUDE,LONGITUDE,LOCATION
921720,4693785,11/09/2016,03/27/2014,RF302041,COMPLIED,11/09/2016,PRV HIGH SIDE,BUILDING,FOR TWO REZNOR 8 TON RTU'S WITH DUAL RECEIVERS...,Install safety valve in high pressure side of ...,...,6033 S COTTAGE GROVE AVE,6033,S,COTTAGE GROVE,AVE,23366,NaN,41.784883,-87.605882,"(41.78488270328645, -87.60588249894568)"


test if unique inspection ids are spread across multiple days

In [35]:
query = f"""
SELECT 
    "INSPECTION NUMBER", 
    COUNT(DISTINCT strptime("VIOLATION DATE", '%m/%d/%Y')) as n_days,
    COUNT(DISTINCT strftime(strptime("VIOLATION DATE", '%m/%d/%Y'), '%Y-%m')) as n_months
FROM df_all
GROUP BY "INSPECTION NUMBER"
HAVING n_months > 1
"""
duckdb.sql(query).df()

,INSPECTION NUMBER,n_days,n_months
0,10278175,2,2
1,10727949,2,2
2,2290066,2,2
3,2272500,3,2
4,10695717,2,2
...,...,...,...
37725,10374141,2,2
37726,14389163,2,2
37727,13073394,2,2
37728,12395971,2,2


In [27]:
df_all["INSPECTION NUMBER"].nunique()

451381

In [36]:
df = df_all[df_all["INSPECTION NUMBER"] == 10278175]
df.head()

,ID,VIOLATION LAST MODIFIED DATE,VIOLATION DATE,VIOLATION CODE,VIOLATION STATUS,VIOLATION STATUS DATE,VIOLATION DESCRIPTION,VIOLATION LOCATION,VIOLATION INSPECTOR COMMENTS,VIOLATION ORDINANCE,...,ADDRESS,STREET NUMBER,STREET DIRECTION,STREET NAME,STREET TYPE,PROPERTY GROUP,SSA,LATITUDE,LONGITUDE,LOCATION
1039749,4346610,12/17/2012,12/17/2012,CN190019,OPEN,NaN,ARRANGE PREMISE INSPECTION,OTHER : :,INTERIOR OF BUILDING NO RESPONSE. UNABLE TO I...,Arrange for inspection of premises. (13-12-100),...,907 W ARGYLE ST,907,W,ARGYLE,ST,11409,34.0,41.973232,-87.652874,"(41.973232363026156, -87.65287395668437)"
1289559,3650234,12/17/2012,10/28/2010,CN140016,NO ENTRY,NaN,DWELLING CLEAN AND SANITARY,INTERIOR:001 :.,"1ST FLOOR CORRIDORS, CARPET IS STAINED THRU OUT .","Keep premises clean, sanitary, and safe. (13-1...",...,907 W ARGYLE ST,907,W,ARGYLE,ST,11409,34.0,41.973232,-87.652874,"(41.973232363026156, -87.65287395668437)"
1289560,3650235,12/17/2012,10/28/2010,CN197087,NO ENTRY,NaN,CARB MONOX DETECT IN RESID,INTERIOR:001 :.,"LAUNDRY ROOM, MISSING CARBON MONOXIDE DETECTOR.",Install carbon monoxide detector within 40 fee...,...,907 W ARGYLE ST,907,W,ARGYLE,ST,11409,34.0,41.973232,-87.652874,"(41.973232363026156, -87.65287395668437)"


# Generate LLM prompts

In [135]:
# WIP
def generate_prompt_from_address(address, df, base_prompt):
    # find the subset of df that matches the address
    df = filter_df_by_address(address, df = df)
    # remove trivial violations
    df = remove_trivial_violations_by_code(df, trivial_codes=TRIVIAL_VIOLATION_CODES)
    # generate narrative report
    narrative_report = generate_narrative_report_for_all_occasions(df)
    # generate prompt
    prompt = base_prompt + "\n" + narrative_report
    return prompt

In [ ]:
df3 = filter_df_by_address("5514 s blackstone", df_recent_subset)
df4 = df3.iloc[0:3, :]
generate_narrative_report_for_one_occasion(df4)

"On 2025-01-02, this property was cited for 3 allegated violations: 1) it allegedly violated city ordiance 'Failed to maintain electric elevator equipment provided at premises in safe and sound working condition. (13-196-590, 13-196-630(b), 18-30-001)'; inspector commented: 'PERFORM CATEGORY 1 TEST ON BOTH ELEVATORS'; 2) it allegedly violated city ordiance 'Failed to maintain electric elevator equipment provided at premises in safe and sound working condition. (13-196-590, 13-196-630(b), 18-30-001)'; inspector commented: 'REPLACE MISSING SMOKE DETECTORS AT 1ST FLOOR ON SOUTH ELEVATOR'; 3) it allegedly violated city ordiance 'Failed to maintain electric elevator equipment provided at premises in safe and sound working condition. (13-196-590, 13-196-630(b), 18-30-001)'; inspector commented: 'UPDATE FIRE EXTINGUISHERS IN BOTH MACHINE ROOMS'; "

In [136]:
out = generate_prompt_from_address("5514 s blackstone", df_recent_subset, BASE_PROMPT)
print(out)


You are a helpful assistant that summarizes long inspection records for an apartment building into a succinct report with a one-line summary as well as bullet points of issues spotted on each occasion. You will be provided a concatenated string of the inspection records, and you will output a single json object that looks like the following example: "

{
  "summary": "This building has received recent complaints regarding <> and <> concerns, including inadequate heat in at least one unit and multiple violations related to <>, which may pose potential <> risks",
  "note": "Some issues on <> are omitted for brevity."
  "summarized_issues": [
    {
      "date": "Jan 2025", // following the Mon YYYY format
      "issues": [
        {
          "emoji": "🧊", // use a relevant emoji and if possible a different one for each issue
          "description": "Insufficient heating (60°F in living room and kitchen) in Unit a, b, c" //
        },
        {
          "emoji": "🚿",
          "descri

In [12]:
def generate_report_from_df(df) -> str:
    out = ""
    for _, row in df.iterrows():
        out += f"""On {row["VIOLATION DATE"].strftime("%Y-%m-%d")}, it received a complaint that it violates city ordinance "{row["VIOLATION DESCRIPTION"]}". The inspector commented: "{row["VIOLATION INSPECTOR COMMENTS"]}". """
    return out

In [15]:
def generate_report_from_address_keyword(kw: str, complaints_only: bool = True) -> str:
    df = filter_df_by_address(kw, complaints_only)
    return generate_report_from_df(df)

In [140]:
address = "5514 S Blackstone"

# Summarize via LLM APIs

In [40]:
from dotenv import load_dotenv
import os

load_dotenv()

openrouter_api_key = os.getenv("OPENROUTER_API_KEY")

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=openrouter_api_key,
)

In [145]:
def clean_json_string(json_string):
    """
    Remove markdown markers i.e. ```json from the json string
    """
    pattern = r"^```json\s*(.*?)\s*```$"
    cleaned_string = re.sub(pattern, r"\1", json_string, flags=re.DOTALL)
    return cleaned_string.strip()

In [178]:
def llm_summarize_from_address_keyword(kw: str) -> dict:
    completion = client.chat.completions.create(
        model="deepseek/deepseek-prover-v2:free",
        response_format={"type": "json_schema"},
        messages=[
            {"role": "user", "content": prompt},
            {"role": "user", "content": generate_report_from_address_keyword(kw)},
        ],
    )
    # raw_return still may contain markdown markers i.e. ```json
    try:
        raw_return = completion.choices[0].message.content
        parsed_return = json.loads(clean_json_string(raw_return))
        return parsed_return
    except Exception as e:
        print(f"Error parsing JSON: {e}")
        return {}

In [158]:
json_eg = llm_summarize_from_address_keyword(address)
print(json_eg)

{'summary': "This building has received complaints regarding inadequate heating, plumbing issues, and structural concerns, posing risks to tenants' comfort and safety. Recent violations highlight heating and water system deficiencies, structural deterioration, and the need for comprehensive repairs and permits.", 'note': 'Some issues which do not directly impact quality of life are omitted for brevity.', 'summarized_issues': [{'date': 'Jan 2025', 'issues': [{'emoji': '❄️', 'description': 'Broken furnace and inadequate heating (66°F) in the building'}, {'emoji': '🛠️', 'description': 'Heating pipes disconnected in the building'}]}, {'date': 'Nov 2024', 'issues': [{'emoji': '🚰', 'description': 'Defective waste piping in Units 6042 (1st and 2nd floors)'}, {'emoji': '🛁', 'description': 'Leaking water from ceiling in bathroom of Unit 6042 (1st floor)'}, {'emoji': '🛀', 'description': 'Insufficient hot water (below 120°F) throughout the building'}]}, {'date': 'Aug 2024', 'issues': [{'emoji': '

In [151]:
json_parsed = clean_json_string(json_eg)
type(json.loads(json_parsed))

dict

In [97]:
# create all unique HP addresses:
all_unique_hp_addresses_with_complaints_since_2020 = complaints["ADDRESS"].unique()
all_unique_hp_addresses_with_complaints_since_2020

array(['6100 S DR MARTIN L KING JR DR', '4838 S CHAMPLAIN AVE',
       '6025 S DR MARTIN L KING JR DR', '1418 E HYDE PARK BLVD',
       '716 E 47TH ST', '1167 E 52ND ST', '6137 S EVANS AVE',
       '6211 S DORCHESTER AVE', '6015 S HARPER AVE',
       '5140 S HYDE PARK BLVD', '860 E 63RD ST', '5118 S DORCHESTER AVE',
       '645 E 62ND ST', '4809 S FORRESTVILLE AVE', '5454 S EVERETT AVE',
       '4916 S DREXEL BLVD', '6201 S CHAMPLAIN AVE', '6241 S DREXEL AVE',
       '6148 S VERNON AVE', '6236 S EVANS AVE', '5445 S INGLESIDE AVE',
       '6045 S DR MARTIN L KING JR DR', '6130 S EBERHART AVE',
       '6200 S ST LAWRENCE AVE', '5124 S GREENWOOD AVE',
       '5540 S HYDE PARK BLVD', '6127 S EVANS AVE', '5464 S ELLIS AVE',
       '6126 S WOODLAWN AVE', '6149 S RHODES AVE', '6132 S EBERHART AVE',
       '6042 S VERNON AVE', '707 E 62ND ST', '4830 S DREXEL BLVD',
       '4940 S EAST END AVE', '5100 S ELLIS AVE',
       '6134 S ST LAWRENCE AVE', '1224 E 52ND ST',
       '5801 S DORCHESTER AVE

In [106]:
# Testing on a few examples
for address in all_unique_hp_addresses_with_complaints_since_2020[:1]:
    summary_json = llm_summarize_from_address_keyword(address)
    print(f"ADDRESS: {address}\nSUMMARY: {summary_json}")
    print("\n----------------------------\n")

ADDRESS: 6100 S DR MARTIN L KING JR DR
SUMMARY: ```json
{
  "summary": "This building has received multiple complaints regarding heating, plumbing, electrical, and structural issues, including inadequate heat, plumbing violations, electrical hazards, and uneven flooring posing trip hazards in various units and common areas",
  "note": "Some issues on denied entry, re-inspection, or deadends are omitted for brevity.",
  "summarized_issues": [
    {
      "date": "Apr 2025",
      "issues": [
        {
          "emoji": "🧊",
          "description": "No entry to basement (possible lack of fire separation)"
        },
        {
          "emoji": "📋",
          "description": "Plans and permits required for repairs or replacement of the floor structure (retail space)"
        },
        {
          "emoji": "🔧",
          "description": "Floors in retail space have damage and require repairs (deflection, broken tiles, damaged subfloors)"
        }
      ]
    },
    {
      "date": "May 

In [179]:
# Load existing data if the file exists
try:
    with open("inspection_summaries.json", "r") as file:
        results = json.load(file)
except FileNotFoundError:
    results = {}

# Load existing addresses with issues if the file exists
try:
    with open("addresses_with_issues.json", "r") as file:
        addresses_with_issues = json.load(file)
except FileNotFoundError:
    addresses_with_issues = []

for idx, address in enumerate(all_unique_hp_addresses_with_complaints_since_2020[64:]):
    print(f"Processing address no. {idx}: {address}")
    try:
        summary_json = llm_summarize_from_address_keyword(address)
        results[address] = summary_json
        # Write the updated results back to the file
        with open("inspection_summaries.json", "w") as file:
            json.dump(results, file, indent=4)
    except Exception as e:
        print(f"Error processing address {address}: {e}")
        addresses_with_issues.append(address)
        continue

# Write the addresses with issues back to the file
with open("addresses_with_issues.json", "w") as file:
    json.dump(addresses_with_issues, file, indent=4)

Processing address no. 0: 5325 S COTTAGE GROVE AVE
Error parsing JSON: 'NoneType' object is not subscriptable
Processing address no. 1: 6040 S HARPER AVE
Processing address no. 2: 4917 S DREXEL BLVD
Processing address no. 3: 727 E 60TH ST
Processing address no. 4: 6158 S DR MARTIN L KING JR DR
Processing address no. 5: 6126 S GREENWOOD AVE
Processing address no. 6: 6211 S KIMBARK AVE
Processing address no. 7: 4900 S DREXEL BLVD
Processing address no. 8: 6253 S COTTAGE GROVE AVE
Processing address no. 9: 6103 S WOODLAWN AVE
Processing address no. 10: 5496 S HYDE PARK BLVD
Processing address no. 11: 5336 S HYDE PARK BLVD
Processing address no. 12: 5130 S DREXEL AVE
Processing address no. 13: 4751 S VINCENNES AVE
Processing address no. 14: 724 E 50TH PL
Processing address no. 15: 6107 S ST LAWRENCE AVE
Processing address no. 16: 5019 S DREXEL BLVD
Error parsing JSON: Expecting value: line 1 column 1 (char 0)
Processing address no. 17: 6206 S RHODES AVE
Processing address no. 18: 6140 S DR

In [168]:
returned = llm_summarize_from_address_keyword("5002 S CHAMPLAIN AVE")

In [180]:
addresses_with_issues

[]